In [ ]:
import pickle
import os
import sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir("../..")

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import pandas as pd
import numpy as np
import seaborn as sns

random_state = 1234

In [ ]:
X = pd.read_csv(os.path.join("notebooks", "feature_importance", "data.csv"), index_col=False)

In [ ]:
X = X[X.columns.difference(['FracInteractionsVal', 'FracInteractionsTest',
       'FracPassiveUsersVal', 'FracPassiveUsersTest', 'FracPassiveItemsVal',
       'FracPassiveItemsTest',])]

In [ ]:
X_train = X[X.columns.difference(['Value','Dataset', 'Method',])].astype(float)
X_train['Dataset'] = X['Dataset']
y_train = X['Value']

In [ ]:
R_x = X_train
R_x['Model_Name'] = X['Method']
R_x['Model_Performance'] = X['Value'].astype(float)


In [ ]:
R_x = R_x[['Model_Name', 'Model_Performance', 'Dataset'] + list(R_x.columns.difference(['Model_Name', 'Model_Performance','Dataset', 'Value'] ))]

In [ ]:
R_x[['Model_Name', 'Model_Performance', 'Dataset', 'APB', 'Density', 'Ginii','Giniu', 'KuPB', 'LTavg']]

,Model_Name,Model_Performance,Dataset,APB,Density,Ginii,Giniu,KuPB,LTavg
0,sasrec,0.015073,brightkite,0.007102,0.001074,0.474072,0.442283,4.133397,0.207341
1,recbole_ItemKNN,0.092339,brightkite,0.007102,0.001074,0.474072,0.442283,4.133397,0.207341
2,recbole_EASE,0.140293,brightkite,0.007102,0.001074,0.474072,0.442283,4.133397,0.207341
3,recbole_MultiVAE,0.118833,brightkite,0.007102,0.001074,0.474072,0.442283,4.133397,0.207341
4,implicit_bpr,0.054292,brightkite,0.007102,0.001074,0.474072,0.442283,4.133397,0.207341
...,...,...,...,...,...,...,...,...,...
177,recbole_EASE,0.065166,ratebeer,0.076873,0.005865,0.750002,0.767440,0.449492,0.145378
178,recbole_MultiVAE,0.057596,ratebeer,0.076873,0.005865,0.750002,0.767440,0.449492,0.145378
179,implicit_bpr,0.058280,ratebeer,0.076873,0.005865,0.750002,0.767440,0.449492,0.145378
180,implicit_als,0.064426,ratebeer,0.076873,0.005865,0.750002,0.767440,0.449492,0.145378


In [ ]:
df = R_x

In [ ]:
import pandas as pd
from itertools import combinations

# # Sample DataFrame
# data = {
#     'Model_Name': ['Model_Name1', 'Model_Name2', 'Model_Name3', 'Model_Name1', 'Model_Name2', 'Model_Name3'],
#     'Model_Performance': [1, 2, 3, 3, 1, 2],
#     'Dataset': ['Dataset1', 'Dataset1', 'Dataset1', 'Dataset2', 'Dataset2', 'Dataset2'],
#     'APB': [1, 1, 1, 2, 2, 2],
#     'Density': [1, 1, 1, 2, 2, 2],
#     'Ginii': [1, 1, 1, 2, 2, 2],
#     'LTavg': [1, 1, 1, 2, 2, 2],
#     'TEST': [1, 1, 1, 3, 3, 3],
# }

# df = pd.DataFrame(data)

# Create a list of unique datasets
datasets = df['Dataset'].unique()

# Get the list of feature columns (excluding 'Model_Name' and 'Model_Performance')
feature_columns = df.columns.difference(['Model_Name', 'Model_Performance', 'Dataset'])

# Initialize an empty DataFrame to store the final result
result_df = pd.DataFrame(columns=['preference'] + feature_columns.tolist())

# Loop through each dataset
for dataset in datasets:
    dataset_df = df[df['Dataset'] == dataset]
    models = dataset_df['Model_Name'].tolist()
    
    # Generate all possible combinations of models
    model_combinations = list(combinations(models, 2))
    
    # Calculate preferences based on Model_Performance
    preferences = []
    for model1, model2 in model_combinations:
        perf1 = dataset_df[dataset_df['Model_Name'] == model1]['Model_Performance'].values[0]
        perf2 = dataset_df[dataset_df['Model_Name'] == model2]['Model_Performance'].values[0]
        if perf1 > perf2:
            preferences.append(f'{model1} > {model2}')
        elif perf1 < perf2:
            preferences.append(f'{model1} < {model2}')
        else:
            preferences.append(f'{model1} = {model2}')
    
    # Calculate mean values for each feature
    feature_means = dataset_df[feature_columns].mean().values.tolist()
    
    # Append the result to the final DataFrame
    result_df = result_df.append({
        'preference': preferences,
        **dict(zip(feature_columns, feature_means))
    }, ignore_index=True)

# Display the final DataFrame
result_df

,preference,APB,Density,Ginii,Giniu,KuPB,LTavg,LTku,LTsk,LTstd,Ni,Nr,Nu,Rpi,Rpu,Shape,SkPB,SpaceSize,StPB
0,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.007102,0.001074,0.474072,0.442283,4.133397,0.207341,0.680672,0.818427,0.164446,14242.0,144069.0,9422.0,10.115784,15.290703,0.661564,1.788994,1.341881e+08,0.006398
1,"[sasrec > recbole_ItemKNN, sasrec < recbole_EA...",0.107248,0.007277,0.842118,0.580842,1.047464,0.130546,2.811047,1.453764,0.129003,12078.0,10027558.0,114090.0,830.233317,87.891647,9.446100,0.915798,1.377979e+09,0.046603
2,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.004125,0.000953,0.530415,0.463656,2.596464,0.197684,1.552966,1.128305,0.164079,19933.0,311252.0,16390.0,15.614910,18.990360,0.822255,1.342222,3.267019e+08,0.002735
3,"[sasrec > recbole_ItemKNN, sasrec > recbole_EA...",0.276871,0.130792,0.598913,0.352268,0.154398,0.280323,0.091844,1.350917,0.186296,8141.0,7640871.0,7176.0,938.566638,1064.781355,0.881464,-1.353652,5.841982e+07,0.054920
4,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.312890,0.130004,0.569654,0.199344,1.619642,0.192479,1.253054,0.398449,0.067959,366.0,50817.0,1068.0,138.844262,47.581461,2.918033,0.063089,3.908880e+05,0.034628
5,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.127566,0.030805,0.660735,0.506666,0.361504,0.166915,1.760237,1.086786,0.115221,3120.0,516312.0,5372.0,165.484615,96.111690,1.721795,0.563471,1.676064e+07,0.037512
6,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.109898,0.009771,0.804832,0.577220,0.905720,0.125709,2.668372,1.398299,0.124506,8263.0,4847239.0,60037.0,586.619751,80.737529,7.265763,0.980782,4.960857e+08,0.048391
7,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.000483,0.000168,0.447626,0.394841,6.550851,0.269711,-0.131500,0.513173,0.189614,58379.0,690996.0,70549.0,11.836380,9.794554,1.208465,2.348454,4.118580e+09,0.000545
8,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.011398,0.002351,0.671934,0.482892,1.409470,0.190712,2.320605,1.217652,0.160647,8753.0,1206261.0,58624.0,137.811150,20.576232,6.697589,0.796955,5.131359e+08,0.004925
9,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.000987,0.000133,0.680637,0.475996,7.228576,0.202586,1.010324,1.073043,0.192853,103427.0,2356699.0,171561.0,22.786110,13.736799,1.658764,2.295436,1.774404e+10,0.000900


In [ ]:
result_df

,preference,APB,Density,Ginii,Giniu,KuPB,LTavg,LTku,LTsk,LTstd,Ni,Nr,Nu,Rpi,Rpu,Shape,SkPB,SpaceSize,StPB
0,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.007102,0.001074,0.474072,0.442283,4.133397,0.207341,0.680672,0.818427,0.164446,14242.0,144069.0,9422.0,10.115784,15.290703,0.661564,1.788994,1.341881e+08,0.006398
1,"[sasrec > recbole_ItemKNN, sasrec < recbole_EA...",0.107248,0.007277,0.842118,0.580842,1.047464,0.130546,2.811047,1.453764,0.129003,12078.0,10027558.0,114090.0,830.233317,87.891647,9.446100,0.915798,1.377979e+09,0.046603
2,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.004125,0.000953,0.530415,0.463656,2.596464,0.197684,1.552966,1.128305,0.164079,19933.0,311252.0,16390.0,15.614910,18.990360,0.822255,1.342222,3.267019e+08,0.002735
3,"[sasrec > recbole_ItemKNN, sasrec > recbole_EA...",0.276871,0.130792,0.598913,0.352268,0.154398,0.280323,0.091844,1.350917,0.186296,8141.0,7640871.0,7176.0,938.566638,1064.781355,0.881464,-1.353652,5.841982e+07,0.054920
4,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.312890,0.130004,0.569654,0.199344,1.619642,0.192479,1.253054,0.398449,0.067959,366.0,50817.0,1068.0,138.844262,47.581461,2.918033,0.063089,3.908880e+05,0.034628
5,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.127566,0.030805,0.660735,0.506666,0.361504,0.166915,1.760237,1.086786,0.115221,3120.0,516312.0,5372.0,165.484615,96.111690,1.721795,0.563471,1.676064e+07,0.037512
6,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.109898,0.009771,0.804832,0.577220,0.905720,0.125709,2.668372,1.398299,0.124506,8263.0,4847239.0,60037.0,586.619751,80.737529,7.265763,0.980782,4.960857e+08,0.048391
7,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.000483,0.000168,0.447626,0.394841,6.550851,0.269711,-0.131500,0.513173,0.189614,58379.0,690996.0,70549.0,11.836380,9.794554,1.208465,2.348454,4.118580e+09,0.000545
8,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.011398,0.002351,0.671934,0.482892,1.409470,0.190712,2.320605,1.217652,0.160647,8753.0,1206261.0,58624.0,137.811150,20.576232,6.697589,0.796955,5.131359e+08,0.004925
9,"[sasrec < recbole_ItemKNN, sasrec < recbole_EA...",0.000987,0.000133,0.680637,0.475996,7.228576,0.202586,1.010324,1.073043,0.192853,103427.0,2356699.0,171561.0,22.786110,13.736799,1.658764,2.295436,1.774404e+10,0.000900


In [ ]:
R_x.to_csv("model_performance.csv", index=False)

In [ ]:
# Pivoting dataframe
df_pivot = R_x.pivot(index='Dataset', columns='Model_Name', values='Model_Performance')

# Generating "preference" string
df_pivot['preference'] = df_pivot.apply(lambda row: "{" + ' > '.join([f"{col}" for col in row.index if not pd.isnull(row[col])]) + "}", axis=1)

# Resetting index and restructuring columns
df_final = R_x.drop(columns=['Model_Name', 'Model_Performance']).drop_duplicates().merge(df_pivot['preference'], on='Dataset')
df_final = df_final[['Dataset', 'preference', 'APB', 'Density', 'Ginii', 'LTavg']]
df_final.columns = ['', 'preference', 'APB', 'Density', 'Ginii', 'LTavg']



In [ ]:
df_final['preference'][0]

'{implicit_als > implicit_bpr > lightfm > recbole_EASE > recbole_ItemKNN > recbole_MultiVAE > sasrec}'